### **Import packages**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

### **Define paths**

In [0]:
silver_path = "abfss://silver@stfinancedev.dfs.core.windows.net/s_branches"
gold_path = "abfss://gold@stfinancedev.dfs.core.windows.net/dim_branches"

### **Create dim_branches delta table schema**

In [0]:
spark.sql(f"""
        CREATE TABLE IF NOT EXISTS finance_cata.gold.dim_branches(
            Branchkey BIGINT GENERATED ALWAYS AS IDENTITY(START WITH 1 INCREMENT BY 1),
            BranchID STRING,
            BankCode STRING,
            BranchName STRING,
            Address STRING,
            City STRING,
            State STRING,
            Pincode STRING,
            IFSC STRING,
            ManagerNAME STRING,
            CreatedAt TIMESTAMP,
            StartDate TIMESTAMP,
            EndDate TIMESTAMP,
            IsActive BOOLEAN
            )USING DELTA
        LOCATION '{gold_path}'
          """)

In [0]:
spark.read.format("delta").load(silver_path).printSchema()

### **Load silver data**

In [0]:
silver_df = spark.read.format("delta").load(silver_path)

### **Add SCD 2 columns**

In [0]:
silver_df = silver_df.withColumn("StartDate", current_timestamp())\
                     .withColumn("EndDate", lit(None).cast("timestamp"))\
                     .withColumn("IsActive", lit(True))

In [0]:
silver_df.display()

### **Load gold data**

In [0]:
if DeltaTable.isDeltaTable(spark, gold_path):
    dim_branches  = DeltaTable.forPath(spark, gold_path)
else:
    # First-time load 
    silver_df.write.format("delta").mode("append").save(gold_path)
    dim_branches = DeltaTable.forPath(spark, gold_path)

### **Delta merge & Load data in gold layer**

In [0]:
(
    dim_branches.alias("t")
    .merge(
        silver_df.alias("s"),
        "t.BranchID = s.BranchID AND t.IsActive = TRUE"
    )
    .whenMatchedUpdate(
        condition = """
        t.BankCode <> s.BankCode OR
        t.BranchName <> s.BranchName OR
        t.Address <> s.Address OR
        t.City <> s.City OR
        t.State <> s.State OR
        t.Pincode <> s.Pincode OR
        t.IFSC <> s.IFSC OR
        t.ManagerName <> s.ManagerName OR
        t.CreatedAt <> s.CreatedAt OR
        """,
        set ={
            "EndDate": "current_date()",
            "IsActive" : "False"
        }
    )
    .whenNotMatchedInsert(
        values={
            "BranchID": "s.BranchID",
            "BankCode": "s.BankCode",
            "BranchName": "s.BranchName",
            "Address": "s.Address",
            "City": "s.City",
            "State": "s.State",
            "Pincode": "s.Pincode",
            "IFSC": "s.IFSC",
            "ManagerName": "s.ManagerName",
            "CreatedAt": "s.CreatedAt",
            "StartDate": "s.StartDate",
            "EndDate": "s.EndDate",
            "IsActive": "s.IsActive"
        }
    )
    .execute()
)

In [0]:
%sql
select * from finance_cata.gold.dim_branches